In [7]:
import os,sys

from keras.models import Model
from keras.layers import Input,LSTM,GRU,Dense,Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt

In [8]:
BATCH_SIZE = 64
EPOCHS = 20
LSTM_NODES =256
NUM_SENTENCES = 20000
MAX_SENTENCE_LENGTH = 50
MAX_NUM_WORDS = 20000
EMBEDDING_SIZE = 100

In [39]:
input_sentences = []
output_sentences = []
output_sentences_inputs = []

count = 0
for line in open('./ben.txt'):
    count += 1

    if count > NUM_SENTENCES:
        break

    if '\t' not in line:
        continue

    spLine = line.rstrip().split('\t')
    input_sentence , output = spLine[0] , spLine[1]
    output_sentence = output + ' <eos>'
    output_sentences_input = '<sos> ' + output

    input_sentences.append(input_sentence)
    output_sentences.append(output_sentence)
    output_sentences_inputs.append(output_sentences_input)

print("num samples input: ",len(input_sentences))
print("num samples output: ",len(output_sentences))
print("num samples input output: ",len(output_sentences_inputs))

num samples input:  4349
num samples output:  4349
num samples input output:  4349


In [40]:
for i in range(5):
    x = np.random.randint(0,4349)
    print(input_sentences[x])
    print(output_sentences[x])
    print(output_sentences_inputs[x])

She was raised in France.
ও ফ্রান্সে বড়ো হয়েছে। <eos>
<sos> ও ফ্রান্সে বড়ো হয়েছে।
I'll never stop.
আমি কখনই থামবো না। <eos>
<sos> আমি কখনই থামবো না।
People from Madrid are weird.
ম্যাড্রিডের মানুষরা অদ্ভুত হয়। <eos>
<sos> ম্যাড্রিডের মানুষরা অদ্ভুত হয়।
I'd like to check out tomorrow morning.
আমি কাল সকালে ঘরটা ছেরে দিতে চাই। <eos>
<sos> আমি কাল সকালে ঘরটা ছেরে দিতে চাই।
She hasn't come yet.
ও এখনো আসেনি। <eos>
<sos> ও এখনো আসেনি।


In [41]:
input_tokenizer = Tokenizer(num_words = MAX_NUM_WORDS)
input_tokenizer.fit_on_texts(input_sentences)
input_integer_seq = input_tokenizer.texts_to_sequences(input_sentences)

word2idx_inputs = input_tokenizer.word_index
print('Total Unique Words: ',len(word2idx_inputs))

max_input_len = max(len(sen) for sen in input_integer_seq)
print('Length of Longest Sequence: ',max_input_len)

Total Unique Words:  1875
Length of Longest Sequence:  19


In [42]:
output_tokenizer = Tokenizer(num_words=MAX_NUM_WORDS, filters='')
output_tokenizer.fit_on_texts(output_sentences + output_sentences_inputs)
output_integer_seq = output_tokenizer.texts_to_sequences(output_sentences)
output_input_integer_seq = output_tokenizer.texts_to_sequences(output_sentences_inputs)

word2idx_outputs = output_tokenizer.word_index
print('Total unique words in the output: %s' % len(word2idx_outputs))

num_words_output = len(word2idx_outputs) + 1
max_out_len = max(len(sen) for sen in output_integer_seq)
print("Length of longest sentence in the output: %g" % max_out_len)

Total unique words in the output: 3551
Length of longest sentence in the output: 19


In [43]:
encoder_input_sequences = pad_sequences(input_integer_seq,maxlen=max_input_len)

print('Input Encoder Shape: ',encoder_input_sequences.shape)
print(encoder_input_sequences[10])

Input Encoder Shape:  (4349, 19)
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 78]


In [44]:
print(word2idx_inputs['i'])
print(word2idx_inputs['will'])

1
73


In [45]:
decoder_input_sequences = pad_sequences(output_input_integer_seq, maxlen=max_out_len, padding='post')
print("decoder_input_sequences.shape:", decoder_input_sequences.shape)
print("decoder_input_sequences[172]:", decoder_input_sequences[172])

decoder_input_sequences.shape: (4349, 19)
decoder_input_sequences[172]: [  2   3  51 835   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0]


In [51]:
print(word2idx_outputs['<sos>'])
print(word2idx_outputs['"আপনি'])
print(word2idx_outputs['একজন'])

2
1107
70


In [52]:
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()

glove_file = open('./utserver/glove.6B.100d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary[word] = vector_dimensions
glove_file.close()
